In [ ]:
import numpy as np 
import pandas as pd 
import os
import glob

import cv2
import albumentations as A 

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import models

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Global variables
IMAGE_SIZE = 512
BATCH_SIZE = 8
LEARNING_RATE = 0.075
LEARNING_RATE_SCHEDULE_FACTOR = 0.1           # Parameter used for reducing learning rate
LEARNING_RATE_SCHEDULE_PATIENCE = 5           # Parameter used for reducing learning rate
IMAGENET_MEAN = [0.485, 0.456, 0.406]         # Mean of ImageNet dataset (used for normalization)
IMAGENET_STD = [0.229, 0.224, 0.225]          # Std of ImageNet dataset (used for normalization)

## Utility functions

def run_length_decode(rle, height=1024, width=1024, fill_value=1):
    component = np.zeros((height, width), np.float32)
    component = component.reshape(-1)
    rle = np.array([int(s) for s in rle.strip().split(' ')])
    rle = rle.reshape(-1, 2)
    start = 0
    for index, length in rle:
        start = start+index
        end = start+length
        component[start: end] = fill_value
        start = end
    component = component.reshape(width, height).T
    return component

## Dataset

In [ ]:
# Read image folder directory
train_dir = '../input/siim-png-images/train_png/'
test_dir = '../input/siim-png-iamges/test_png/'
mask_dir = '../input/siim-mask-png/mask_png'

# Read dataframe from csv file
train_df = pd.read_csv('../input/siim-segment-csv/train.csv')
val_df = pd.read_csv('../input/siim-segment-csv/val.csv')

In [ ]:
# Define list of image transformations
def get_transforms(phase, size=IMAGE_SIZE, mean=IMAGENET_MEAN, std=IMAGENET_STD, normalization=False):
    # Define list of image transformations
    image_transformation = [A.Resize(height=size, width=size, p=1.0)]
        
    if(phase == 'train'):
        image_transformation.append(A.HorizontalFlip(p=0.5))
        image_transformation.append(A.OneOf([A.RandomContrast(), A.RandomGamma(), A.RandomBrightness(),], p=0.3))
        # image_transformation.append(A.OneOf([A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        #                             A.GridDistortion(), A.OpticalDistortion(distort_limit=2, shift_limit=0.5)], p=0.3))
        image_transformation.append(A.ShiftScaleRotate(shift_limit=0.01, scale_limit=0.3, rotate_limit=10, border_mode=cv2.BORDER_CONSTANT, p=0.4))
    
    if normalization:
            image_transformation.append(A.Normalize(mean=mean, std=std)) # Normalization with mean and std from ImageNet
            
    image_transformation = A.Compose(image_transformation)
    
    return image_transformation

In [ ]:
# Impelement Dataset Loader
class SIIMDataset(Dataset):
    
    def __init__(self, df, folder_dir, mask_dir, phase, normalization):
        self.df = df
        self.folder_dir = folder_dir
        self.mask_dir = mask_dir
        self.aug_transform = get_transforms(phase, normalization=normalization)
        self.positive_num = len(self.df[self.df['has_mask'] == 1])
        self.negative_num = len(self.df) - self.positive_num
        self.labels = []
        
        for index in range(len(self.df)):
            self.labels.append(self.df['has_mask'][index])
            
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        image_id = self.df['ImageId'][index]
        image_path = os.path.join(self.folder_dir, image_id + ".png")
        img_data = cv2.imread(image_path)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB) # Convert image to RGB channels
        
        image_label = [self.df['has_mask'][index]] 
        if image_label[0] == 1:
            mask_path = os.path.join(self.mask_dir, image_id + ".png")
            mask_data = cv2.imread(mask_path)[:, :, 0]
        else: 
            mask_data = np.zeros([1024, 1024])
        mask_data = (mask_data/255).astype(np.float32)
        
        augmented_data = self.aug_transform(image=img_data, mask=mask_data) # Apply transformation to image and mask
        img_data = augmented_data['image']
        mask_data = augmented_data['mask']
        
        return torch.FloatTensor(img_data), torch.FloatTensor(image_label), torch.FloatTensor(mask_data) # Covert image and mask to torch tensor

In [ ]:
# Create training dataset and training data loader
train_dataset = SIIMDataset(train_df, train_dir, mask_dir, phase='train', normalization=True)
train_dl = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle = True, num_workers=8, pin_memory=True)

val_dataset = SIIMDataset(val_df, train_dir, mask_dir, phase='val', normalization=True)
val_dl = DataLoader(dataset = val_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers=8, pin_memory=True)

# Show images and masks in first batch of train dataloader
def show_aug(inputs, nrows=1, ncols=8, image=True):
    plt.figure(figsize=(3*ncols, 3*nrows))
    plt.subplots_adjust(wspace=0., hspace=0.)
    i_ = 0
    
    if len(inputs) > 8:
        inputs = inputs[:8]
        
    for idx in range(len(inputs)):
        img = inputs[idx].numpy().astype(np.float32)
        
        plt.subplot(nrows, ncols, i_+1)
        if image:
            plt.imshow(img[:,:,0], cmap='bone')
        else:
            plt.imshow(img)
        plt.axis('off')
        
        i_ += 1
 
    return plt.show()

    
images, _, masks = next(iter(train_dl))

show_aug(images)
show_aug(masks, image=False)

## Define network and architecture

In [ ]:
# Get backbone model and its needed layers' information
def get_backbone(name, pretrained=False):

    """ Loading backbone, defining names for skip-connections and encoder output. """
    
    # Loading backbone model
    if name == 'resnet18':
        backbone = models.resnet18(pretrained=pretrained)
    elif name == 'resnet34':
        backbone = models.resnet34(pretrained=pretrained)
    elif name == 'resnet50':
        backbone = models.resnet50(pretrained=pretrained)
    elif name == 'resnet101':
        backbone = models.resnet101(pretrained=pretrained)
    elif name == 'resnet152':
        backbone = models.resnet152(pretrained=pretrained)
    elif name == 'vgg16':
        backbone = models.vgg16_bn(pretrained=pretrained).features
    elif name == 'vgg19':
        backbone = models.vgg19_bn(pretrained=pretrained).features
    elif name == 'densenet121':
        backbone = models.densenet121(pretrained=pretrained).features
    elif name == 'densenet161':
        backbone = models.densenet161(pretrained=pretrained).features
    elif name == 'densenet169':
        backbone = models.densenet169(pretrained=pretrained).features
    elif name == 'densenet201':
        backbone = models.densenet201(pretrained=pretrained).features
    elif name == 'unet_encoder':
        from unet_backbone import UnetEncoder
        backbone = UnetEncoder(3)
    else:
        raise NotImplemented('{} backbone model is not implemented so far.'.format(name))

    # Specifying skip feature and output names
    if name.startswith('resnet'):
        feature_names = [None, 'relu', 'layer1', 'layer2', 'layer3']
        backbone_output = 'layer4'
    elif name == 'vgg16':
        feature_names = ['5', '12', '22', '32', '42']
        backbone_output = '43'
    elif name == 'vgg19':
        feature_names = ['5', '12', '25', '38', '51']
        backbone_output = '52'
    elif name.startswith('densenet'):
        feature_names = [None, 'relu0', 'denseblock1', 'denseblock2', 'denseblock3']
        backbone_output = 'denseblock4'
    elif name == 'unet_encoder':
        feature_names = ['module1', 'module2', 'module3', 'module4']
        backbone_output = 'module5'
    else:
        raise NotImplemented('{} backbone model is not implemented so far.'.format(name))

    return backbone, feature_names, backbone_output

In [ ]:
# Implements upsample blocks in UNet architecture
class UpsampleBlock(nn.Module):
    
    def __init__(self, ch_in, ch_out=None, skip_in=0):
        """
        Init upsample block architecture
        
        Parameters
        ----------
        ch_in: int
            number of input's channels
        ch_out: int
            number of output's channels
        skip_in: 
            number of skip connection input's channels
        """
        super().__init__()
        
        # Define upsample block layers:
        
        # Up convolution
        self.up_conv = nn.ConvTranspose2d(in_channels=ch_in, out_channels=ch_out, kernel_size=(4, 4),
                                          stride=2, padding=1, output_padding=0, bias=False) 
        self.bn = nn.BatchNorm2d(ch_out)
        self.skipbn = nn.BatchNorm2d(skip_in)
        
        # First convolution
        conv1_in = skip_in + ch_out 
        self.conv1 = nn.Conv2d(in_channels=conv1_in, out_channels=ch_out, kernel_size=(3, 3),
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(ch_out)
        
        # Second convolution
        conv2_in = ch_out 
        self.conv2 = nn.Conv2d(in_channels=conv2_in, out_channels=ch_out, kernel_size=(3, 3),
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(ch_out)
        
        # ReLU layer
        self.relu = nn.ReLU(inplace=True)
        
        # Dropout layer
        # self.dropout = nn.Dropout(p=0.5, inplace=True)
        
    def forward(self, x, skip_connection):
        # print('Input:', x.shape)
        x = self.up_conv(x)
        x = self.bn(x)
        x = self.relu(x)
        # print('After upconv:', x.shape)
        
        if skip_connection is not None:
            skip_connection = self.skipbn(skip_connection)
            skip_connection = self.relu(skip_connection)
            # print('Skip connection:', skip_connection.shape)
            x = torch.cat([x, skip_connection], dim=1)
            # x = self.dropout(x)
            # print("After concat:", x.shape, '\n')
        
        # Double convolution
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        return x

In [ ]:
# Implements upsample blocks in UNet architecture
class UpsampleBlockWithSERes(nn.Module):
    
    def __init__(self, ch_in, ch_out=None, skip_in=0):
        """
        Init upsample block architecture
        
        Parameters
        ----------
        ch_in: int
            number of input's channels
        ch_out: int
            number of output's channels
        skip_in: 
            number of skip connection input's channels
        """
        super().__init__()
        
        # Define upsample block layers:
        
        # Up convolution
        self.up_conv = nn.Sequential(
            nn.ConvTranspose2d(in_channels=ch_in, out_channels=ch_out, kernel_size=(4, 4), 
                               stride=2, padding=1, output_padding=0, bias=False),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )
        
        self.skipbn = nn.BatchNorm2d(skip_in)
        
        # First convolution (bottleneck) 
        conv1_in = skip_in + ch_out 
        self.bottle_neck = nn.Sequential(
            nn.Conv2d(in_channels=conv1_in, out_channels=ch_out, kernel_size=(1, 1),stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )
        
        # Residual module
        self.res = nn.Sequential(
            nn.Conv2d(in_channels=ch_out, out_channels=ch_out, kernel_size=(3, 3), stride=1, padding=1, bias=False),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=ch_out, out_channels=ch_out, kernel_size=(1, 1), stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )
        
        # Squeeze and excitation module
        r = 8
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(ch_out, int(ch_out/r), bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(int(ch_out/r), ch_out, bias=False),
            nn.Sigmoid()
        )
    
        
    def forward(self, x, skip_connection):
        # Up convolution
        # print('Input:', x.shape)
        x = self.up_conv(x)
        # print('After upconv:', x.shape)
        
        if skip_connection is not None:
            skip_connection = self.skipbn(skip_connection)
            skip_connection = nn.ReLU(inplace=True)(skip_connection)
            # print('Skip connection:', skip_connection.shape)
            x = torch.cat([x, skip_connection], dim=1)
            # x = self.dropout(x)
            # print("After concat:", x.shape, '\n')
        
        # Pass through bottleneck
        bottle_neck_output = self.bottle_neck(x)
        
        # Pass through residual module
        res_output = self.res(bottle_neck_output)
        # print(res_output.shape)
        
        # Passs through squeeze and excitation module
        se_output = self.se(res_output)
        
        # Merge output of residual module and se module (multiplication)
        x = torch.mul(res_output, se_output.unsqueeze(-1).unsqueeze(-1))
        
        # Add skip connection from bottle neck
        x = x + bottle_neck_output
        
        del skip_connection, bottle_neck_output, res_output, se_output
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        
        return x

In [ ]:
# Implement UNet with backbone model (two classes)
class UNet(nn.Module):
    
    def __init__(self, 
                 backbone_name='densenet121', 
                 pretrained=False, 
                 encoder_freeze=False,
                 upsample_out_chs=(1024, 512, 256, 64, 32)):
        
        """
        Init UNet architecture with backbone model
        
        Parameters
        ----------
        backbone_name: string
            name of backbone model 
        pretrained: bool
            whether using pretrained model from ImageNet or not
        encoder_freeze: bool 
            whether freezing encoder's parameters
        decoder_filters: tuple
            tuple contains decoder filters' size
        """
        
        super().__init__()
        
        # Get backbone model information
        self.backbone, self.skip_feature_names, self.bb_out_name = get_backbone(backbone_name, pretrained=pretrained)
        skip_chs, bb_out_ch = self.infer_skip_channels() # Getting the number of channels at skip connections and at the output of the encoder
        if encoder_freeze: 
            self.freeze_encoder() # Freezing encoder parameters
        
        # Build upsample component
        self.upsample_component = nn.ModuleList() # List of upsample blocks
        upblock_out_chs = upsample_out_chs[:len(self.skip_feature_names)]  # Avoiding having more blocks than skip connections
        upblock_in_chs = [bb_out_ch] + list(upblock_out_chs[:-1]) # Number of decoder filters' input channels
        num_blocks = len(self.skip_feature_names) # Number of upsample block
        
        for i, [upblock_in_ch, upblock_out_ch] in enumerate(zip(upblock_in_chs, upblock_out_chs)):
            # print('upsample_blocks[{}] in: {}  skip: {}  out: {}'.format(i, upblock_in_ch, skip_chs[num_blocks-i-1], upblock_out_ch))
            self.upsample_component.append(UpsampleBlockWithSERes(upblock_in_ch, upblock_out_ch, skip_in=skip_chs[num_blocks-i-1]))
            
        # Final convolution layer
        self.final_conv = nn.Conv2d(upblock_out_chs[-1], 1, kernel_size=(1, 1)) # Two classes segmentation
        self.sigmoid = nn.Sigmoid()
        
    
    def infer_skip_channels(self):
        
        """ Getting the number of channels at skip connections and at the output of the encoder. """
        
        x = torch.zeros(1, 3, IMAGE_SIZE, IMAGE_SIZE)
        skip_chs = [0] 

        # forward run in backbone to count channels 
        for name, child in self.backbone.named_children():
            x = child(x)
            if name in self.skip_feature_names:
                skip_chs.append(x.shape[1])
            if name == self.bb_out_name:
                bb_out_ch = x.shape[1]
                break
                
        return skip_chs, bb_out_ch
    
    
    def freeze_encoder(self):

        """ Freezing encoder parameters, the newly initialized decoder parameters are remaining trainable. """

        for param in self.backbone.parameters():
            param.requires_grad = False
    
    
    def forward_backbone(self, x):
        
        """ Forward propagation in backbone encoder network.  """
        
        features = {None:None}
        for name, child in self.backbone.named_children():
            x = child(x)
            if name in self.skip_feature_names:
                features[name] = x
            if name == self.bb_out_name: 
                break
        
        return x, features
    
    def forward(self, *input):
        
        """ Forward propagation in U-Net. """

        x, features = self.forward_backbone(*input)
        
        for skip_feature_name, upsample_block in zip(self.skip_feature_names[::-1], self.upsample_component):
            skip_feature = features[skip_feature_name]
            x = upsample_block(x, skip_feature)
        
        x = self.final_conv(x)
        x = self.sigmoid(x)
        
        return x

## Define loss and metric

In [ ]:
# Loss function
def Dice_Loss(preds, targets):
    assert(preds.shape == targets.shape)
    smooth = 0.1
    intersection = 2.0 * ((targets * preds).sum(axis=(1,2,3))) + smooth
    union = preds.sum(axis=(1,2,3)) + targets.sum(axis=(1,2,3)) + smooth

    return 1 - (intersection / union)

def Dice_BCE_Loss(preds, targets):
    dice_loss = Dice_Loss(preds, targets)
    bce_loss = nn.BCELoss()
    bce_loss = bce_loss(preds, targets)
    
    return dice_loss.mean() + bce_loss

# Define loss function for model
loss_func = nn.BCELoss()

In [ ]:
# Metric
def IoU(preds, targets, eps=1e-5):
    intersection = (targets * preds).sum(axis=(1,2,3))
    union = targets.sum(axis=(1,2,3)) + preds.sum(axis=(1,2,3)) - intersection
    return ((intersection+eps)/(union+eps)).mean()

def Dice(preds, targets, eps=1e-5):
    intersection = 2*(targets * preds).sum(axis=(1,2,3))
    union = targets.sum(axis=(1,2,3)) + preds.sum(axis=(1,2,3)) 
    return ((intersection+eps)/(union+eps)).mean()

# Define metric for model
metric = IoU

## Training model

In [ ]:
# Check GPU available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Load model from saved checkpoint
def load_model_from_checkpoint(filepath, encoder_freeze=False):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    
    for parameter in model.parameters():
        parameter.requires_grad = True
    
    if encoder_freeze:
        for param in model.backbone.parameters():
            param.requires_grad = False
    
    return model

# Learning rate range test (Linear)
def lr_range_test(train_dl, device, loss_func, base_lr, end_lr, num_iter, num_epoch):
    """
    Learning rate range test

    Paramteters
    -----------
    train_dl: Dataset
      data loader for training
    device: str
      "cpu" or "cuda"
    loss_func: loss function
      loss function used for training
    base_lr: float
        base learning rate
    end_lr: float
        end learning rate
    num_iter: int
        number of iterations to update learning rate
    num_epoch: int
        number of epochs for implementing learning range test

    Returns
    -------
    list
      learning rates and training losses
    """
    
    # model = load_model_from_checkpoint('../input/siim-segment-cp/checkpoint_57eps.pth', False)
    model = UNet(backbone_name='densenet121', pretrained=True, 
                 encoder_freeze=False, upsample_out_chs=(128, 64, 32, 16, 8)).to(device) # Init model to eval (UNet)
    optimizer = optim.SGD(model.parameters(), lr=base_lr, momentum=0.9, nesterov=True, weight_decay=1e-6) # Optimizer
    total_iter = num_epoch * len(train_dl) # Total number of iterations for training num_epoch
    losses = [] # List of training loss
    lrs = []    # List of learning rate
    training_loss = 0 # Total training loss of num_iter iterations
    
    for epoch in range(num_epoch):
        
        # Upsample minority class
        df = train_df
        df_minor = df[df['has_mask']==1]
        df_major = df[df['has_mask']==0]
        df_minor_upsample = resample(df_minor, replace=True, n_samples = len(df_major), random_state=69)
        df = pd.concat([df_major, df_minor_upsample]).reset_index()
        train_dataset.df = df
        
        for index, (inputs, _, targets) in enumerate(train_dl):
            if (epoch * len(train_dl) + index) % num_iter == 0: 
                training_loss = 0
            # Move X, Y  to device (GPU)
            inputs = inputs.permute(0,3,1,2).to(device)
            targets = targets.unsqueeze(1).to(device)
            
            # Clear previous gradient
            optimizer.zero_grad()

            # Feed forward the model
            preds = model(inputs)
            
            # Caculate loss
            loss = loss_func(preds, targets)
            training_loss += loss

            # Back propagation
            loss.backward()

            # Update parameters
            optimizer.step()
            
            del inputs, targets, preds
            if torch.cuda.is_available(): torch.cuda.empty_cache()
            
            # Update learning rate
            if (epoch * len(train_dl) + index) % num_iter == num_iter - 1:
                # Update list of learning rates
                lrs.append(optimizer.param_groups[0]['lr']) 
                
                # Update list of training losses
                losses.append(training_loss/num_iter) 
                print(optimizer.param_groups[0]['lr'], training_loss/num_iter)
                
                # Increase learning rate linearly between base_lr and end_lr after num_iter iterations
                #lr = base_lr * (end_lr/base_lr)**((epoch * len(train_dl) + index) / total_iter) # Exponential
                lr = base_lr + (end_lr - base_lr)*((epoch * len(train_dl) + index) / total_iter) 
                
                # Update current learning rate of optimizer
                optimizer.param_groups[0]['lr'] = lr 
                
    return lrs, losses

# Perform learning rate range test (Linear)
num_epoch = 2
num_iter = 100
base_lr = 1e-6
end_lr = 0.1
lrs, losses = lr_range_test(train_dl, device, loss_func, base_lr, end_lr, num_iter, num_epoch)
# print (lrs,'\n')
# print (losses)

# Create model and check model architecture
# model = load_model_from_checkpoint('../input/siim-segment-checkpoint/checkpoint_57eps.pth', True)
model = UNet(backbone_name='densenet121', pretrained=True, encoder_freeze=False, upsample_out_chs=(128, 64, 32, 16, 8)).to(device) 
# print(model)
# upsample_out_chs=(1024, 512, 256, 64, 32)

# Check number of trainable parameters
sum(p.numel() for p in model.parameters() if p.requires_grad)

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, nesterov=True, weight_decay=1e-6)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=LEARNING_RATE_SCHEDULE_FACTOR, 
                                                 patience=LEARNING_RATE_SCHEDULE_PATIENCE, verbose=True)

In [ ]:
# Training one epoch on training dataset
def epoch_training(epoch, model, train_dl, device, loss_func, metric, optimizer, sigmoid_threshold=0.5):
    """
    Epoch training

    Paramteters
    -----------
    epoch: int
      epoch number
    model: torch Module
      model to train
    train_dl: Dataset
      data loader for training
    device: str
      "cpu" or "cuda"
    loss_func: loss function
      loss function used for training
    optimizer: torch optimizer
      optimizer used for training
    metric: metric function
        metric function used for evaluating
    sigmoid_threshold: float
        threshold for sigmoid function
    
    Returns
    -------
    float
      training loss
    float
      train_iou
    """
    # Switch model to training mode
    model.train()
    
    training_loss = 0 # Storing sum of training losses
    train_iou = 0 # Storing sum of training iou
   
    # For each batch
    for (inputs, _, targets) in train_dl:
        
        # Move X, Y  to device (GPU)
        inputs = inputs.permute(0,3,1,2).to(device)
        targets = targets.unsqueeze(1).to(device)
        
        # Clear previous gradient
        optimizer.zero_grad()

        # Feed forward the model
        preds = model(inputs)
        loss = loss_func(preds, targets)

        # Back propagation
        loss.backward()

        # Update parameters
        optimizer.step()

        # Update training loss after each batch
        training_loss += loss.item()
        
        # Caculate metric score
        preds = (preds >= sigmoid_threshold).float().to(device)
        train_iou += metric(preds, targets).item()
    
    # Clear memory
    del inputs, targets, preds
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    
    # return training loss and metric score
    return training_loss/len(train_dl), train_iou/len(train_dl) 

In [ ]:
# Evaluate model in the validation dataset
def evaluating(epoch, model, val_dl, device, loss_func, metric, optimizer, sigmoid_threshold=0.5):
    """
    Validate model on validation dataset
    
    Parameters
    ----------
    epoch: int
        epoch number
    model: torch Module
        model used for validation
    val_dl: Dataset
        data loader of validation set
    device: str
        "cuda" or "cpu"
    loss_func: loss function
      loss function used for training
    optimizer: torch optimizer
      optimizer used for training
    metric: metric function
        metric function used for evaluating
    sigmoid_threshold: float
        threshold for sigmoid function
  
    Returns
    -------
    float
        loss on validation set
    float
        val_iou
    """

    # Switch model to evaluation mode
    model.eval()

    val_loss = 0    # Total loss of model on validation set
    val_iou  = 0    # Total iou of model on validation set
    
    with torch.no_grad(): # Turn off gradient
        # For each batch
        for (inputs, _, targets) in val_dl:
        
            # Move X, Y  to device (GPU)
            inputs = inputs.permute(0,3,1,2).to(device)
            targets = targets.unsqueeze(1).to(device)

            # Feed forward the model
            preds = model(inputs)
            loss = loss_func(preds, targets)
            
            # Update validation loss after each batch
            val_loss += loss.item()
            
            # Caculate metric score
            preds = (preds >= sigmoid_threshold).float().to(device)
            val_iou += metric(preds, targets).item()
    
    # Clear memory
    del inputs, targets, preds
    if torch.cuda.is_available(): torch.cuda.empty_cache()
        
    # return validation loss and metric score
    return val_loss/len(val_dl), val_iou/len(val_dl)

# Initialize before training
train_losses = []
val_losses = []
train_ious = []
val_ious = []

In [ ]:
# Define loading function from saving filepath, lock backbone
def load_checkpoint(filepath, encoder_freeze=False):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']
    scheduler = checkpoint['scheduler']
    train_losses = checkpoint['train_losses']
    val_losses = checkpoint['val_losses']
    train_ious = checkpoint['train_ious']
    val_ious = checkpoint['val_ious']
    
    for parameter in model.parameters():
        parameter.requires_grad = True
        
    if encoder_freeze:
        for param in model.backbone.parameters():
            param.requires_grad = False

    return model, optimizer, scheduler, train_losses, val_losses, train_ious, val_ious

In [ ]:
## Load from checkpoint 
model, optimizer, scheduler, train_losses, val_losses, train_ious, val_ious = load_checkpoint("../input/siim-segment-checkpoint/checkpoint_20eps.pth")
model.to(device)

# Fully training the model

# Training each epoch
for epoch in range(7):
    # Upsample minority class
    df = train_df
    df_minor = df[df['has_mask']==1]
    df_major = df[df['has_mask']==0]
    df_minor_upsample = resample(df_minor, replace=True, n_samples = len(df_major), random_state=69)
    df = pd.concat([df_major, df_minor_upsample]).reset_index()
    train_dataset.df = df    
    
    # Training
    train_loss, train_iou = epoch_training(epoch, model, train_dl, device, loss_func, metric, optimizer)
    train_losses.append(train_loss)
    train_ious.append(train_iou)

    # Evaluating
    val_loss, val_iou = evaluating(epoch, model, val_dl, device, loss_func, metric, optimizer)
    val_losses.append(val_loss)
    val_ious.append(val_iou)
    
    # Update learning rate
    scheduler.step(train_loss)
    
    # Define checkpoint
    checkpoint = {'model': model,
                  'optimizer': optimizer,
                  'scheduler': scheduler,
                  'train_losses': train_losses,
                  'val_losses': val_losses,
                  'train_ious': train_ious,
                  'val_ious': val_ious}

    # Save model
    torch.save(checkpoint, '/kaggle/working/checkpoint_'+str(epoch+29)+'eps.pth')
    
    print("Epoch:", epoch+29, '\n', 
          "Train loss:", train_loss, "T-IoU:", train_iou, '\n',
          "Val loss:", val_loss, "V-IoU:" , val_iou, '\n')

# Plot Validation IoU vs Training IoU in saved checkpoint
plt.figure(figsize=(10,5))
x = [i for i in range(1,51)]
print(train_ious)
print(val_ious)
plt.plot(x, train_ious, 'b', label='Training IoU')
plt.plot(x, val_ious, 'r', label='Validation IoU')
# ticks = [x for x in range(1,31)]
# plt.xticks(ticks, ticks)
plt.title('IoU and epoch')
plt.xlabel('Epoch')
plt.ylabel('IoU')
plt.legend()
plt.show()

In [ ]:
# Inspect information from saved epoch
print('Min train losses:', min(train_losses), 'at epoch: ', train_losses.index(min(train_losses))+1)
print('Min val losses:', min(val_losses), 'at epoch: ', val_losses.index(min(val_losses))+1)
print('Max train iou:', max(train_ious), 'at epoch: ', train_ious.index(max(train_ious))+1)
print('Max val iou:', max(val_ious), 'at epoch: ', val_ious.index(max(val_ious))+1)

## Post processing

In [ ]:
# Post processing with connected components
def pp_with_connected_components(pred):
    pred = (pred.squeeze(1).squeeze(0)>=0.5).float()
    pred = pred.data.cpu().numpy().astype(np.uint8)
        
    _, thresh = cv2.threshold(pred,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    num_labels, labels , stats, centroids = cv2.connectedComponentsWithStats(thresh , connectivity , cv2.CV_32S)
        
    for idx, stat in enumerate(stats):
        x, y, w, h, c = stat
            
        if w == pred.shape[1] or h == pred.shape[0]:
            continue
            
        THRESHOLD = 900
            
        if c < THRESHOLD:
            pred[labels == idx] = 0
            
    pred = torch.FloatTensor(pred).unsqueeze(0).unsqueeze(1)
    
    return pred

# Post processing with connected components and evaluation in validation set 
connectivity = 8  # You need to choose 4 or 8 for connectivity type
sigmoid_threshold = [0.1, 0.3, 0.5, 0.7]
val_dice  = [0, 0, 0 ,0]    # Total dice of model on validation set
val_iou = [0, 0, 0, 0]      # Total iou of model on validation set
post_processing = True
model.eval()

with torch.no_grad():
    for index in range(len(val_dataset)):
        image, _, target = val_dataset[index]
        image = image.unsqueeze(0).permute(0,3,1,2).to(device)
        target = target.unsqueeze(0).unsqueeze(1).to(device)
        pred = model(image)
        
        if post_processing:
            pred = pp_with_connected_components(pred)
        
        # Caculate metric score
        for i in range(len(sigmoid_threshold)):
            pred_thres = (pred >= sigmoid_threshold[i]).float().to(device)
            val_dice[i] += Dice(pred_thres, target).item()
            val_iou[i] += IoU(pred_thres, target).item()
    
for i in range(len(sigmoid_threshold)):
    print('Threshold: ', sigmoid_threshold[i], 'IoU: ', val_iou[i]/len(val_dataset), 'Dice: ', val_dice[i]/len(val_dataset))

In [ ]:
# Test time augmentation and dis-augmentation
def test_time_aug(input_img):
    aug_list = []
    
    trans_list = [A.HorizontalFlip(p=1)]
    
    for transform in trans_list:
        augmented_data = transform(image=input_img.numpy())
        img_data = augmented_data['image']
        aug_list.append(torch.FloatTensor(img_data))

    return aug_list

def test_time_dis_aug(aug_pred_list):
    dis_aug_pred_list = []
    
    dis_trans_list = [A.HorizontalFlip(p=1)]
    
    for i in range(len(aug_pred_list)):
        augmented_data = dis_trans_list[i](image=aug_pred_list[i].squeeze(1).squeeze(0).cpu().numpy())
        img_data = augmented_data['image']
        dis_aug_pred_list.append(img_data)
        
    return dis_aug_pred_list

In [ ]:
# Evaluate validation set with test time augmentation
model.eval()

sigmoid_threshold = [0.1, 0.3, 0.5, 0.7]
val_dice  = [0, 0, 0 ,0]    # Total dice of model on validation set
val_iou = [0, 0, 0, 0]      # Total iou of model on validation set
    
with torch.no_grad(): # Turn off gradient
    # For each batch
    for index in range(len(val_dataset)):
        image, _, target = val_dataset[index]
        target = target.unsqueeze(0).unsqueeze(1).to(device)
        
        # Test time augmentation
        aug_list = test_time_aug(image)

        # Feed forward the model to prediction
        pred = model(image.unsqueeze(0).permute(0,3,1,2).to(device)).squeeze(1).squeeze(0)
    
        aug_pred_list = []
        for aug_image in aug_list:
            aug_pred_list.append(model(aug_image.unsqueeze(0).permute(0,3,1,2).to(device)))
        
        # Dis_augmentation
        dis_aug_pred_list = test_time_dis_aug(aug_pred_list)
        dis_aug_pred_list.append(pred.squeeze(1).squeeze(0))
    
        # Merge
        pred = torch.FloatTensor(dis_aug_pred_list).mean(axis=0).to(device).unsqueeze(0).unsqueeze(1)
        
        # Caculate metric score
        for i in range(len(sigmoid_threshold)):
            pred_thres = (pred >= sigmoid_threshold[i]).float().to(device)
            val_dice[i] += Dice(pred_thres, target).item()
            val_iou[i] += IoU(pred_thres, target).item()
    
for i in range(len(sigmoid_threshold)):
    print('Threshold: ', sigmoid_threshold[i], 'IoU: ', val_iou[i]/len(val_dataset), 'Dice: ', val_dice[i]/len(val_dataset))

def show_aug(inputs, nrows=1, ncols=8, image=True):
    plt.figure(figsize=(3*ncols, 3*nrows))
    plt.subplots_adjust(wspace=0., hspace=0.)
    i_ = 0
    
    if len(inputs) > 8:
        inputs = inputs[:8]
        
    for idx in range(len(inputs)):
        img = inputs[idx].numpy().astype(np.float32)
        
        plt.subplot(nrows, ncols, i_+1)
        if image:
            plt.imshow(img[:,:,0], cmap='bone')
        else:
            plt.imshow(img)
        plt.axis('off')
        
        i_ += 1
 
    return plt.show()

def show_aug(inputs, nrows=1, ncols=8, image=True):
    plt.figure(figsize=(3*ncols, 3*nrows))
    plt.subplots_adjust(wspace=0., hspace=0.)
    i_ = 0
    
    if len(inputs) > 8:
        inputs = inputs[:8]
        
    for idx in range(len(inputs)):
        img = inputs[idx].numpy().astype(np.float32)
        
        plt.subplot(nrows, ncols, i_+1)
        if image:
            plt.imshow(img[:,:,0], cmap='bone')
        else:
            plt.imshow(img)
        plt.axis('off')
        
        i_ += 1
 
    return plt.show()

# Show some model predictions
images, _, masks = next(iter(val_dl))

model.eval()
with torch.no_grad():
    inputs = images.permute(0,3,1,2).to(device)
    preds = model(inputs)
    preds = ((preds.squeeze(1))>=0.5).float()
    preds = preds.cpu()
    
show_aug(images)
show_aug(preds, image=False)
show_aug(masks, image=False)